# Reading in data

In [2]:
!wget http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip

--2021-11-05 14:05:04--  http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip
Resolving www2.informatik.uni-freiburg.de (www2.informatik.uni-freiburg.de)... 132.230.105.133
Connecting to www2.informatik.uni-freiburg.de (www2.informatik.uni-freiburg.de)|132.230.105.133|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘BX-CSV-Dump.zip.3’

BX-CSV-Dump.zip.3   100%[===================>]  24.88M  15.8MB/s    in 1.6s    

2021-11-05 14:05:08 (15.8 MB/s) - ‘BX-CSV-Dump.zip.3’ saved [26085508/26085508]



In [3]:
!unzip BX-CSV-Dump.zip

Archive:  BX-CSV-Dump.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Book-Ratings.csv     
replace BX-Books.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Books.csv            
replace BX-Users.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: BX-Users.csv            


In [3]:
import pandas as pd

In [4]:
users = pd.read_csv('BX-Users.csv')

ParserError: ignored

In [5]:
!head BX-Users.csv

"User-ID";"Location";"Age"
"1";"nyc, new york, usa";NULL
"2";"stockton, california, usa";"18"
"3";"moscow, yukon territory, russia";NULL
"4";"porto, v.n.gaia, portugal";"17"
"5";"farnborough, hants, united kingdom";NULL
"6";"santa monica, california, usa";"61"
"7";"washington, dc, usa";NULL
"8";"timmins, ontario, canada";NULL
"9";"germantown, tennessee, usa";NULL


In [6]:
users = pd.read_csv('BX-Users.csv', sep=';')

UnicodeDecodeError: ignored

In [7]:
users = pd.read_csv('BX-Users.csv', sep=';', encoding = "ISO-8859-1")
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [8]:
books = pd.read_csv('BX-Books.csv', sep=';', encoding = "ISO-8859-1")

ParserError: ignored

In [9]:
!head -n6452 BX-Books.csv | tail -n1

"0735201994";"Peterman Rides Again: Adventures Continue with the Real \"J. Peterman\" Through Life &amp; the Catalog Business";"John Peterman";"2000";"Prentice Hall Art";"http://images.amazon.com/images/P/0735201994.01.THUMBZZZ.jpg";"http://images.amazon.com/images/P/0735201994.01.MZZZZZZZ.jpg";"http://images.amazon.com/images/P/0735201994.01.LZZZZZZZ.jpg"


In [10]:
books = pd.read_csv('BX-Books.csv', sep=';', encoding = "ISO-8859-1", escapechar = "\\")
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [11]:
ratings = pd.read_csv('BX-Book-Ratings.csv', sep=';', encoding = "ISO-8859-1")
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


# Collaborative filtering

## Transforming user ratings into utility matrix

In [ ]:
# do not try to run, you will run out of memory
# ratings_matrix = ratings.pivot(index='User-ID', columns='ISBN', values='Book-Rating').fillna(0)

In [12]:
ratings.shape

(1149780, 3)

In [13]:
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

In [14]:
sparce_matrix = csr_matrix((ratings['Book-Rating'], (ratings['User-ID'], ratings['ISBN'])))

TypeError: ignored

In [15]:
# inspired by https://stackoverflow.com/a/53235048
def create_matrix(frame, user_col, item_col, rating_col):
    """
    creates the sparse user-item interaction matrix

    Parameters
    ----------
    frame : DataFrame
        implicit rating data

    user_col : str
        user column name

    item_col : str
        item column name

    ratings_col : str
        implicit rating column name
    """

    user_c = CategoricalDtype(sorted(frame[user_col].unique()), ordered=True)
    item_c = CategoricalDtype(sorted(frame[item_col].unique()), ordered=True)

    row = frame[user_col].astype(user_c).cat.codes
    col = frame[item_col].astype(item_c).cat.codes
    return csr_matrix((frame[rating_col], (row, col)), \
                              shape=(user_c.categories.size, item_c.categories.size))

In [16]:
sparse_matrix = create_matrix(ratings, 'User-ID', 'ISBN', 'Book-Rating')
sparse_matrix

<105283x340556 sparse matrix of type '<class 'numpy.longlong'>'
	with 1149780 stored elements in Compressed Sparse Row format>